# RL Lecture 4: Model-free prediction

Prediction = Policy evaluation

Notes from a Reinformcement Course by David Silver (https://www.youtube.com/watch?v=PnHCvfgC_ZA&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ&index=4)

We are getting rid of the assumption, that we know the Markov Descision Process. In particular, we don't know what the reward is in each state and what are the probabilities what state we are going to be at after taking an action and even what the states are exactly.

## Easy21
Assignment for a Reinforcement Learning Course, mentioned at 16:15.

### 1. Implementation of Easy21

In [1]:
import random

In [70]:
def draw_card():
    u = random.random()
    color = 'black' if u < (2/3) else 'red'
    new_card = random.randint(1, 10)
    return new_card if 'black' else -new_card

class State:
    def __init__(self, dealer_value=None, player_value=None):
        self.dealer = dealer_value or random.randint(1, 10)
        self.player = player_value or 0
        
    def hit(self):
        new_card = draw_card()
        return State(self.dealer, self.player + new_card)
        
    def __str__(self):
        return f'State(dealer={self.dealer}, player={self.player})'
    
    def __repr__(self):
        return str(self)

In [75]:
def step(state, action):
    """Function simulating Easy21 evironment. Returns (state: State, reward: int)"""
    if action == 'stick':  # let the dealer play
        while state.dealer < 17:
            state.dealer += draw_card()
        if state.dealer > 21:
            return state, 1
        elif state.dealer > state.player:
            return state, -1
        else:
            return state, 0
    else:  # action == 'hit'
        state = state.hit()
        if state.player > 21:
            return state, -1
        else:
            return state, 0

### 2. Monte-Carlo Control in Easy21